In [1]:
import numpy as np
import pandas as pd
from numpy import dstack
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt


def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values



# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset//')
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset//')
    
    # #zero-offset class values
    # trainy = trainy - 1
    # testy = testy - 1
    # #one hot encode y
    # trainy_one_hot = to_categorical(trainy)
    # testy_one_hot = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [2]:
###Loading Data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 1) (2947, 128, 9) (2947, 1)


In [4]:
from gaf import GAF

In [5]:
gf1=GAF.transform(trainX[0])

In [6]:
gf1.shape

(1152, 1152)

array([[ 1.00000000e+00, -1.23216700e-01,  1.02934100e-01, ...,
         2.88178100e-02, -3.77180000e-05, -1.44600600e-03],
       [-1.23216700e-01, -9.69635290e-01, -9.99791644e-01, ...,
        -9.95518470e-01, -9.92375140e-01, -9.92200579e-01],
       [ 1.02934100e-01, -9.99791644e-01, -9.78809142e-01, ...,
        -9.91308729e-01, -9.94692060e-01, -9.94835981e-01],
       ...,
       [ 2.88178100e-02, -9.95518470e-01, -9.91308729e-01, ...,
        -9.98339068e-01, -9.99585767e-01, -9.99625306e-01],
       [-3.77180000e-05, -9.92375140e-01, -9.94692060e-01, ...,
        -9.99585767e-01, -9.99999997e-01, -9.99998899e-01],
       [-1.44600600e-03, -9.92200579e-01, -9.94835981e-01, ...,
        -9.99625306e-01, -9.99998899e-01, -9.99995818e-01]])

In [11]:
gf2=GAF.transform(trainX[1])

In [20]:
gf=np.empty((0,1152,1152))
gf=np.dstack((gf1,gf2),axis=1)

TypeError: _dstack_dispatcher() got an unexpected keyword argument 'axis'

In [18]:
gf.shape

(1152, 1152, 2)

In [24]:
gaf_trainShape=trainX.shape[1]*trainX.shape[2]
gaf_train=np.empty((0,gaf_trainShape,gaf_trainShape))
for i in range(81):
    gaf=GAF.transform(trainX[i])#transformig each timeseries of train data into a Gram Matrix
    gaf_train=np.vstack([gaf_train,[gaf]])

In [25]:
gaf_train.shape 

(81, 1152, 1152)

In [27]:
gaf_testShape=testX.shape[1]*testX.shape[2]
gaf_test=np.empty((0,gaf_testShape,gaf_testShape))
for i in range(21):
    gaf=GAF.transform(testX[i])#transformig each timeseries of test data into a Gram Matrix
    gaf_test=np.vstack([gaf_test,[gaf]])

In [28]:
gaf_test.shape

(21, 1152, 1152)